<a href="https://colab.research.google.com/github/verdwis/Text-Data-Augmentation-With-EDA/blob/main/Easy_Data_Augmentation_(EDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import peckage

In [14]:
import random
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet as wn
import spacy
from spacy.lang.fr.examples import sentences
from spacy.lang.en.examples import sentences
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# load dataset

In [74]:
df = pd.read_csv('/content/drive/MyDrive/Tesis_Very_2022/EDA/Dataset_EDA/preprocessing.csv')
df.head(10)

,tweets,label
0,poles tanjung balai laksana vaksinasi,1
1,dukung program presiden jokong juta vaksinasi ...,1
2,panglima tni marsekal tni had tjahjanto sebut ...,1
3,polka lampung gelar vaksinasi solid untuk angg...,1
4,bupati kobar apresiasi usaha laksana vaksinasi...,1
5,ps columba asia indonesia dukung program vaksi...,1
6,kam laksana vaksinasi bagi warga lanjut usia d...,1
7,kabupaten di kabar sebut tak serius vaksinasi ...,1
8,salah banget vaksinasi belum uji menang,0
9,silah simak jadwal vaksinasi per lurah ikut in...,1


# cek distribusi data

In [75]:
df.label.value_counts()

1    77
0    17
2     6
Name: label, dtype: int64

# inisialisasi data yang akan di augmentasi

In [76]:
# define a dataframe of only negative reviews
df_positiv = df[df['label']==2]

In [77]:
df_positiv.head()

,tweets,label
13,masalah awak tak faham lembaga dan jaya tindak...,2
14,dr najihatussalehah ahmad kata raja telah bukt...,2
22,johor bahri jun hibah ion jabat di kampung mes...,2
29,takma satu pun pilih hati kalau ada pilih semu...,2
55,fokus sampai maklumat lanjut laksana perintah ...,2


In [78]:
df_positiv.to_csv('/content/df_positiv.csv', index=False)

# 1. Synonym Replacement.

In [79]:
from nltk.corpus import wordnet

def get_synonyms(word):
    
    synonyms = set()
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [80]:
from nltk.corpus import stopwords
stop_words = []
for w in stopwords.words('indonesian'):
    stop_words.append(w)
print(stop_words)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [81]:
import random

In [82]:

def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

In [83]:
print(f" Example of Synonym Replacement: {synonym_replacement('hey man how are you doing',3)}")

 Example of Synonym Replacement: hey serviceman how equal you answer


# 2. Random Deletion

In [84]:
def random_deletion(words, p):

    words = words.split()
    
    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    sentence = ' '.join(new_words)
    
    return sentence

# 3. Random Swap

In [85]:
def swap_word(new_words):
    
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        
        if counter > 3:
            return new_words
    
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

# This will Swap the words

In [86]:
def random_swap(words, n):
    
    words = words.split()
    new_words = words.copy()
    # n is the number of words to be swapped
    for _ in range(n):
        new_words = swap_word(new_words)
        
    sentence = ' '.join(new_words)
    
    return sentence

# 4. Random Insertion

In [87]:
def random_insertion(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        add_word(new_words)
        
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    
    synonyms = []
    counter = 0
    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


In [88]:
df_positiv.head()

,tweets,label
13,masalah awak tak faham lembaga dan jaya tindak...,2
14,dr najihatussalehah ahmad kata raja telah bukt...,2
22,johor bahri jun hibah ion jabat di kampung mes...,2
29,takma satu pun pilih hati kalau ada pilih semu...,2
55,fokus sampai maklumat lanjut laksana perintah ...,2


# Build Augmentation

In [89]:
def create_augmentation(data, alpha, operation):
    n = len(data)
    data_copy = data.copy()
    for i in range(n):
        if operation.lower() == 'sr':
            data_copy.at[i, 'tweets'] = synonym_replacement(data.at[i, 'tweets'], alpha)
        if operation.lower() == 'ri':
            data_copy.at[i, 'tweets'] = random_insertion(data.at[i, 'tweets'], alpha)
        if operation.lower() == 'rs':
            data_copy.at[i, 'tweets'] = random_swap(data.at[i, 'tweets'], alpha)
        if operation.lower() == 'rd':
            data_copy.at[i, 'tweets'] = random_deletion(data.at[i, 'tweets'], alpha)
    return data_copy

In [90]:
def augment_data(data, k, alpha, operation):
    data_copy = data.copy()
    for i in range(k):
        df = create_augmentation(data, alpha, operation)
        data_copy = pd.concat([data_copy, df], ignore_index = True)
    return data_copy.drop_duplicates()

In [91]:
tes = pd.read_csv('/content/df_positiv.csv')
tes.head(5)

,tweets,label
0,masalah awak tak faham lembaga dan jaya tindak...,2
1,dr najihatussalehah ahmad kata raja telah bukt...,2
2,johor bahri jun hibah ion jabat di kampung mes...,2
3,takma satu pun pilih hati kalau ada pilih semu...,2
4,fokus sampai maklumat lanjut laksana perintah ...,2


In [98]:
# augment the data 2 times using synonym replacement operations
eda_sr = augment_data(tes, 2, 0.2, 'sr')

In [99]:
eda_sr.head()

,tweets,label
0,masalah awak tak faham lembaga dan jaya tindak...,2
1,dr najihatussalehah ahmad kata raja telah bukt...,2
2,johor bahri jun hibah ion jabat di kampung mes...,2
3,takma satu pun pilih hati kalau ada pilih semu...,2
4,fokus sampai maklumat lanjut laksana perintah ...,2


In [101]:
eda_sr.label.value_counts()

2    16
Name: label, dtype: int64

# concat dataset

In [105]:
eda_sr = pd.concat([eda_sr, df], ignore_index=True, sort=False)

In [106]:
eda_sr.head()

,tweets,label
0,masalah awak tak faham lembaga dan jaya tindak...,2
1,dr najihatussalehah ahmad kata raja telah bukt...,2
2,johor bahri jun hibah ion jabat di kampung mes...,2
3,takma satu pun pilih hati kalau ada pilih semu...,2
4,fokus sampai maklumat lanjut laksana perintah ...,2


In [107]:
eda_sr.label.value_counts()

1    77
2    22
0    17
Name: label, dtype: int64

# Save Dataset

In [108]:
eda_sr.to_csv('/content/eda_sr.csv', index=False)